<div style="text-align: center; background-color: #559cff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
 Final Project -Programming for  Data Science @ FIT-HCMUS, VNU-HCM 📌
</div>

<div style="text-align: center; font-family: 'Trebuchet MS', Arial, sans-serif; padding: 20px; font-size: 25px; font-weight: bold; border-radius: 0 0 0 0">
  TOPIC: ANALYSIS ON GOOGLE PLAY STORE APP
</div>

STUDENT INFORMATION:


OVERVIEW OF THE TOPIC:
- Reason forchoosing topic:


DATASET INFORMATION:
- Source:
- License: 

# Data Exploration - Preprocessing


## Import Library


In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):

## Exploring data

### Read raw data from file

In [2]:
apps_df = pd.read_csv("../Data/Raw/googleplaystore.csv")

### How many rows and how many columns?

In [3]:
# Shape of dateset
shape = apps_df.shape
print(f'Number of rows: {shape[0]}')
print(f'Number of columns: {shape[1]}')

Number of rows: 10840
Number of columns: 13


### What is the meaning of each row?

In [4]:
apps_df.head(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


Each row contains information of an application on Google Play Store.

### Are there duplicated rows?

In [5]:
# index = apps_df.index
detectDupSeries = apps_df.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()
print(f'The data have {num_duplicated_rows} duplicated rows')

The data have 483 duplicated rows


So we're going to drop duplicated rows.

In [6]:
apps_df.drop_duplicates(keep='first', inplace=True)
apps_df.shape

(10357, 13)

### What is the meaning of each column?

**Apps dataset columns descriptions:**

This file contains all the details of the apps on Google Play. There are 13 features that describe a given app.

- **App**: Name of the app
- **Category**: Category of the app. Some examples are: ART_AND_DESIGN, FINANCE, COMICS, BEAUTY etc.
- **Rating**: The current average rating (out of 5) of the app on Google Play.
- **Reviews**: Number of user reviews given on the app.
- **Size**: Size of the app in MB (megabytes).
- **Installs**: Number of times the app was downloaded from Google Play
- **Type**: Whether the app is paid or free.
- **Price**: Price of the app in US$
- **Last Updated**: Date on which the app was last updated on Google Play
- **Content Rating**: Subject the app aim to.
- **Genres**: Type of the application.
- **Current Ver**: Newest version that has been released by the publisher.
- **Android Ver**: The cofiguration that devices need in order to work properly.rk properly.roperly.yd or fr Google Play

###  What is the current data type of each column? Are there columns having inappropriate data types?

In [7]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews             int64
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

We observe that `Installs`, `Price`, `Size` should have *numeric* datatype and `Last Updated` should has *datetime* datatype.

`Installs` feature also should has numeric datatype. First, we will remove non-digit charater in this columns, then, convert them to numeric.

In [8]:
apps_df['Installs'] = apps_df['Installs'].str.replace(r'\D', '', regex=True)
apps_df['Installs'] = pd.to_numeric(apps_df['Installs'], errors='raise')

In `Price` column, first, we'll find out what make them be non-numeric.

In [9]:
price_series = apps_df['Price']
strange_data = price_series[~price_series.str.isnumeric()]
strange_data

234       $4.99 
235       $4.99 
427       $3.99 
476       $3.99 
477       $6.99 
          ...   
10734     $0.99 
10759     $7.99 
10781    $16.99 
10784     $1.20 
10797     $1.04 
Name: Price, Length: 765, dtype: object

At first, we predict that the '$' at the head makes them being strings. Eliminating this character does not affecting our dataset, so we should remove them. However, to be more carefully, we would check whether there are any non-number characters in this column. 

In [10]:
strange_data[~strange_data.str.contains(r'\$')]

Series([], Name: Price, dtype: object)

Now, we see all problems that make the column "non-numeric". We'll handle them.

In [11]:
apps_df['Price'] = apps_df['Price'].str.replace(r'\$', '', regex=True)
apps_df['Price'] = pd.to_numeric(apps_df['Price'], errors='raise')

Now, we would convert `Last Updated` to *datetime*.

In [12]:
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], format="%d-%b-%y", errors='raise')

To pre-process `Size`, we would split the size number and their unit to new column. Then, we would scale them in to the same unit (M) because some apps have size value in k instead of M

In [13]:
def SizeUnit(str): # return the unit of size
    if str[-1] == 'M':
        return 'M'
    elif str[-1] == 'k':
        return 'k'
    elif str == 'Varies with device':
        return 'Varies with device'

# create new column to store size unit
apps_df['Size Unit'] = apps_df['Size'].apply(lambda x: SizeUnit(x))
# remove unit attached to size
apps_df['Size'] = apps_df['Size'].apply(lambda x: x[:-1])
# replace unclear size by np.nan
apps_df['Size'] = apps_df['Size'].replace('Varies with devic', np.nan)

# change all value to the same scale
apps_df['Size'] = pd.to_numeric(apps_df['Size'], errors='raise')
apps_df['Size'] = np.where(apps_df['Size Unit'] == 'k', apps_df['Size']/1024, apps_df['Size'])

# drop 'Size Unit' because it is no longer neccessary
apps_df.drop(columns='Size Unit', inplace=True)

Double check on data type.

In [14]:
apps_df.dtypes

App                       object
Category                  object
Rating                   float64
Reviews                    int64
Size                     float64
Installs                   int64
Type                      object
Price                    float64
Content Rating            object
Genres                    object
Last Updated      datetime64[ns]
Current Ver               object
Android Ver               object
dtype: object

Now, they seem appropriate for analysing!!!

### With each numerical column, how are values distributed?

In [20]:
num_col_info_df = apps_df.select_dtypes(include='number')

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

num_col_info_df = num_col_info_df.agg([missing_ratio, 'mean', 'min', lower_quartile, median, upper_quartile, 'max'])
num_col_info_df

,Rating,Reviews,Size,Installs,Price
missing_ratio,0.00000,0.000000e+00,0.000000,0.000000e+00,0.0000
mean,4.18638,4.059046e+05,20.147291,1.415776e+07,1.0308
min,1.00000,0.000000e+00,0.008301,0.000000e+00,0.0000
lower_quartile,4.00000,3.200000e+01,5.400000,1.000000e+03,0.0000
median,4.30000,1.680000e+03,12.000000,1.000000e+05,0.0000
upper_quartile,4.50000,4.641600e+04,27.000000,1.000000e+06,0.0000
max,5.00000,7.815831e+07,100.000000,1.000000e+09,400.0000


The data have nothing abnormal. However, there are some missing values with small ratio - smaller than 20%, so we're going to fill them.
- For `Rating`, we would fill them with the mean of each `Category`, because the difference between *mean* and *median* is not significant.
- For `Size`, we would fill them with the median of each `Category`, because the *mean* is very different from the *median*.

In [16]:
apps_df['Rating'] = apps_df.groupby('Category')['Rating'].transform(lambda x: x.fillna(x.mean()))
apps_df['Size'] = apps_df.groupby('Category')['Size'].transform(lambda x: x.fillna(x.median()))

In [18]:
apps_df.isna().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       8
Android Ver       2
dtype: int64

In [19]:
apps_df.describe(include='all')

/var/folders/5s/3t7kh6rn1_5_jk7f9tjf_zrh0000gn/T/ipykernel_96461/1514725508.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  apps_df.describe(include='all')


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,10357,10357,10357.000000,1.035700e+04,10357.000000,1.035700e+04,10356,10357.000000,10357,10357,10357,10349,10355
unique,9659,33,NaN,NaN,NaN,NaN,2,NaN,6,119,1377,2783,33
top,ROBLOX,FAMILY,NaN,NaN,NaN,NaN,Free,NaN,Everyone,Tools,2018-08-03 00:00:00,Varies with device,4.1 and up
freq,9,1943,NaN,NaN,NaN,NaN,9591,NaN,8382,842,298,1302,2379
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-05-21 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-08 00:00:00,NaN,NaN
mean,NaN,NaN,4.186380,4.059046e+05,20.147291,1.415776e+07,NaN,1.030800,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.485246,2.696778e+06,21.284104,8.023955e+07,NaN,16.278625,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,0.000000e+00,0.008301,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.047411,3.200000e+01,5.400000,1.000000e+03,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
